In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3 as lite
from sqlite3 import Error
from pathlib import Path
from datetime import date
import numpy as np
import seaborn as sns
import matplotlib.ticker as tick
import requests
import difflib as diff
import re 
import csv
import ast
%matplotlib inline
import sqlite3

In [2]:
conn = sqlite3.connect('cvefixes.db')

In [3]:
query = """
SELECT * from 
"""

In [4]:
query_one = """
SELECT cv.cvss3_base_score,cv.exploitability_score,cv.impact_score,cv.cvss2_base_score, f.filename,f.change_type,f.nloc,f.programming_language,f.token_count,f.complexity, f.num_lines_added, f.num_lines_deleted, f.code_before, f.code_after,fx.hash,fx.repo_url, cc.cwe_id,cc.cve_id,cw.description 
FROM file_change f, commits c, fixes fx, cve cv, cwe_classification cc,cwe cw
WHERE f.hash = c.hash 
AND c.hash = fx.hash 
AND cc.cwe_id = cw.cwe_id
AND fx.cve_id = cv.cve_id 
AND cv.cve_id = cc.cve_id 
AND f.num_lines_added<=1 
AND f.num_lines_deleted<=1;
"""

In [5]:
df = pd.read_sql_query(query_one, conn)

In [6]:
df.head(5)

,cvss3_base_score,exploitability_score,impact_score,cvss2_base_score,filename,change_type,nloc,programming_language,token_count,complexity,num_lines_added,num_lines_deleted,code_before,code_after,hash,repo_url,cwe_id,cve_id,description
0,9.8,3.9,5.9,7.5,Makefile,ModificationType.MODIFY,nan,Shell,nan,nan,1,1,"# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...","# Copyright (C) 1991,92,93,94,95,96,97,98,99 F...",2864e767053317538feafa815046fff89e5a16be,https://github.com/bminor/glibc,CWE-252,CVE-1999-0199,The software does not check the return value f...
1,nan,nan,nan,4.6,saver.cpp,ModificationType.MODIFY,197,C++,1132,34,1,0,#include <pwd.h>\n#include <unistd.h>\n#includ...,#include <pwd.h>\n#include <unistd.h>\n#includ...,04906bd5de2f220bf100b605dad37b4a1d9a91a6,https://github.com/KDE/kde1-kdebase,NVD-CWE-Other,CVE-1999-0731,NVD is only using a subset of CWE for mapping ...
2,nan,nan,nan,4.0,svr_principal.c,ModificationType.MODIFY,1655.0,C,10633.0,439.0,1,1,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,/* -*- mode: c; c-basic-offset: 4; indent-tabs...,c5be6209311d4a8f10fda37d0d3f876c1b33b77b,https://github.com/krb5/krb5,NVD-CWE-Other,CVE-2012-1013,NVD is only using a subset of CWE for mapping ...
3,nan,nan,nan,7.6,spnego_mech.c,ModificationType.MODIFY,3104.0,C,15617.0,512.0,0,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",f18ddf5d82de0ab7591a36e465bc24225776940f,https://github.com/krb5/krb5,CWE-415,CVE-2014-4343,The product calls free() twice on the same mem...
4,nan,nan,nan,7.8,spnego_mech.c,ModificationType.MODIFY,3104.0,C,15621.0,513.0,1,1,"/*\n * Copyright (C) 2006,2008 by the Massachu...","/*\n * Copyright (C) 2006,2008 by the Massachu...",524688ce87a15fc75f87efc8c039ba4c7d5c197b,https://github.com/krb5/krb5,CWE-476,CVE-2014-4344,A NULL pointer dereference occurs when the app...


In [7]:
df.replace("nan", np.nan, inplace=True)
nan_counts = df.isna().sum()
nan_counts

cvss3_base_score        1001
exploitability_score    1001
impact_score            1001
cvss2_base_score         115
filename                   0
change_type                0
nloc                    2133
programming_language       0
token_count             2133
complexity              2133
num_lines_added            0
num_lines_deleted          0
code_before                0
code_after                 0
hash                       0
repo_url                   0
cwe_id                     0
cve_id                     0
description                0
dtype: int64

In [8]:
file ='dataset.csv'

In [9]:
df.to_csv(file,index=False)